### 1. Load Packages

In [1]:
import openai
import pandas as pd
import re
import json

### 2. Load Tokens

In [2]:
openai.api_key = "To be filled"

### 3. Load Data or Text

In [3]:
df1 = pd.read_csv('processed_60900_61899.csv')
df2 = pd.read_csv('processed_177000_177999.csv')

In [4]:
df1

,Unnamed: 0,GLOBAL_ID,LOCAL_ID,processed_txt,date,_date_,WoY
0,60900,60900,6555,we already have congestion pricing for taxis &...,2021-07-09 00:02:34+00:00,2021-07-09,2021-27
1,60901,60901,6556,an the best cannot be that much i am sorry htt...,2021-07-08 23:44:03+00:00,2021-07-08,2021-27
2,60902,60902,6557,handicap uber's is getting out of hand,2021-07-08 23:40:10+00:00,2021-07-08,2021-27
3,60903,60903,6558,bitchh i would calmly get in a cab laugh ill b...,2021-07-08 23:16:21+00:00,2021-07-08,2021-27
4,60904,60904,6559,take a cab. you are getting hepatitis walking ...,2021-07-08 23:15:04+00:00,2021-07-08,2021-27
...,...,...,...,...,...,...,...
995,61895,61895,7550,so...i confess ...i walked to target &amp; bou...,2021-06-17 23:27:09+00:00,2021-06-17,2021-24
996,61896,61896,7551,"uber, ferry, plane, train today",2021-06-17 23:00:01+00:00,2021-06-17,2021-24
997,61897,61897,7552,"hi, i order some food about a hour ago and i h...",2021-06-17 22:08:29+00:00,2021-06-17,2021-24
998,61898,61898,7553,i was there. it was an uber. none of the other...,2021-06-17 21:54:38+00:00,2021-06-17,2021-24


In [5]:
df2

,Unnamed: 0,GLOBAL_ID,LOCAL_ID,processed_txt,date,_date_,WoY
0,177000,177000,7102,please kick japan out of my country they are s...,2021-11-26 13:17:13+00:00,2021-11-26,2021-47
1,177001,177001,7103,once i learned about correctional exercises an...,2021-11-26 12:41:27+00:00,2021-11-26,2021-47
2,177002,177002,7104,empty train cars &gt;&gt;,2021-11-26 12:22:52+00:00,2021-11-26,2021-47
3,177003,177003,7105,that emerald parking lot is about 1 mile walk ...,2021-11-26 12:14:05+00:00,2021-11-26,2021-47
4,177004,177004,7106,"that is going to take time, i need car by end ...",2021-11-26 11:48:54+00:00,2021-11-26,2021-47
...,...,...,...,...,...,...,...
995,177995,177995,8097,how about getting all trash off the sidewalk? ...,2021-06-03 02:03:54+00:00,2021-06-03,2021-22
996,177996,177996,8098,get trash off the sidewalk. trash corals in th...,2021-06-03 02:00:17+00:00,2021-06-03,2021-22
997,177997,177997,8099,people of philadelphia find trae young's hotel...,2021-06-03 01:55:00+00:00,2021-06-03,2021-22
998,177998,177998,8100,ring rain coming back for more and do not you ...,2021-06-03 01:40:50+00:00,2021-06-03,2021-22


### 4.Chain of thought

In [6]:
def generate_chain_of_thought(tweet):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages = [
            {
                "role": "system",
                "content": "You are an AI specifically trained to extract travel experiences from tweets, focusing on the mode of transportation, the sentiment towards it, and the reasons behind the sentiment. It's crucial to maintain consistency in your analysis, avoiding assumptions not supported by the tweet's content (overfitting), yet also ensuring to provide as much detail as possible without defaulting to 'na' unless absolutely necessary (underfitting). Your responses should be in lowercase and adhere to the specified categories."
            },
            {
                "role": "user",
                "content": f"Analyze the tweet: '{tweet.lower()}'. Your goal is to clearly identify the travel experience it describes, focusing on these aspects:\n\n"
                   f"Given the tweet: \"{prepared_tweet}\" 1. What is the corresponding travel mode? (Train, subway, bus, bike, taxi, Uber, private car) 2. Is the user satisfied with the corresponding travel mode’s service? (Yes or No) 3. If not satisfied, what are the reasons behind?"
                   "Conclude with a concise, structured summary (keep this format for all your response) in JSON format, like this: "
                   "{\"travel_mode\": \"<inferred mode>\", \"sentiment\": \"<inferred sentiment>\", \"reasons\": \"<inferred reasons>\"}. Remember, consistency and adherence to the evidence presented in the tweet are key."
            }
        ],
        temperature=0.7,  # A slightly higher temperature for creativity in judgment
        max_tokens=250
    )
    content = response.choices[0].message.content if response.choices else "No response"
    return content
    

In [7]:
def process_tweet_and_generate_output(df):
    processed_data = []
    
    for index, row in df.iterrows():
        analysis_result_json = generate_chain_of_thought(row['processed_txt'])
        
        try:
            # Attempt to parse the JSON string
            analysis_result = json.loads(analysis_result_json)
        except json.JSONDecodeError:
            # If parsing fails, use default values
            analysis_result = {"travel_mode": "na", "sentiment": "neutral", "reasons": "Parsing error or invalid format in response."}
        
        # Extract and store the relevant information along with the GLOBAL_ID and original tweet
        processed_row = {
            "GLOBAL_ID": row['GLOBAL_ID'],
            "Tweet": row['processed_txt'],
            "Travel Mode": analysis_result.get('travel_mode', 'NA'),
            "Sentiment": analysis_result.get('sentiment', 'NA'),
            "Reasons": analysis_result.get('reasons', 'NA')
        }
        
        # Append the processed row to the list of processed data
        processed_data.append(processed_row)

    # Convert the processed data into a DataFrame
    return pd.DataFrame(processed_data)

### 5.Outcome

**(1) df1: processed_60900_61899.csv**

In [8]:
# test
process_tweet_and_generate_output(df1.iloc[0:2])

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons
0,60900,we already have congestion pricing for taxis &...,car,positive,congestion pricing for cars and trucks is inte...
1,60901,an the best cannot be that much i am sorry htt...,na,na,na


In [9]:
processed_df1 = process_tweet_and_generate_output(df1)
processed_df1.to_csv('result_df1_edit.csv', index=False)

In [11]:
processed_df1

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons
0,60900,we already have congestion pricing for taxis &...,car,positive,congestion pricing is intended to encourage pe...
1,60901,an the best cannot be that much i am sorry htt...,na,na,The tweet does not provide clear information r...
2,60902,handicap uber's is getting out of hand,car,negative,The sentiment is negative towards Uber for the...
3,60903,bitchh i would calmly get in a cab laugh ill b...,taxi,positive,The author expresses a positive sentiment towa...
4,60904,take a cab. you are getting hepatitis walking ...,taxi,negative,The sentiment towards taking a taxi is negativ...
...,...,...,...,...,...
995,61895,so...i confess ...i walked to target &amp; bou...,uber,positive,The sentiment towards the Uber ride is positiv...
996,61896,"uber, ferry, plane, train today",plane,neutral,the tweet mentions various modes of transporta...
997,61897,"hi, i order some food about a hour ago and i h...",food delivery,negative,delayed food delivery
998,61898,i was there. it was an uber. none of the other...,taxi,neutral,the author mentions taking an Uber but doesn't...


In [15]:
processed_df1['Travel Mode'].value_counts().get('na', 0)

253

**(2) df2: processed_177000_177999.csv**

In [13]:
processed_df2 = process_tweet_and_generate_output(df2)
processed_df2.to_csv('result_df2_edit.csv', index=False)

In [14]:
processed_df2

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons
0,177000,please kick japan out of my country they are s...,car,negative,The sentiment towards the car transportation m...
1,177001,once i learned about correctional exercises an...,car,neutral,The tweet metaphorically compares individuals ...
2,177002,empty train cars &gt;&gt;,train,positive,The author expresses a positive sentiment towa...
3,177003,that emerald parking lot is about 1 mile walk ...,car,negative,nightmare leaving the lot if you do not leave ...
4,177004,"that is going to take time, i need car by end ...",car,positive,The author expresses the need for a car by the...
...,...,...,...,...,...
995,177995,how about getting all trash off the sidewalk? ...,car,negative,The sentiment towards the car transportation m...
996,177996,get trash off the sidewalk. trash corals in th...,car,negative,The author expresses frustration with trash co...
997,177997,people of philadelphia find trae young's hotel...,car,negative,The sentiment is negative as the author expres...
998,177998,ring rain coming back for more and do not you ...,car,positive,"The sentiment towards the car is positive, lik..."


In [16]:
processed_df2['Travel Mode'].value_counts().get('na', 0)

49

### 6. Verfication

In [24]:
def verify_llm_output(tweet, travel_mode, sentiment, reasons):
    verification_prompt = f"""
    Based on the following tweet analysis:
    Tweet: {tweet}
    - Travel Mode: {travel_mode}
    - Sentiment: {sentiment}
    - Reasons: {reasons}

    Is this analysis 'correct' or 'incorrect'? Consider the relevance and accuracy of the travel mode, sentiment, and reasons given.
    """
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{
            "role": "system",
            "content": "You're a highly intelligent assistant. Please read the following analysis carefully and provide your verdict."
        }, {
            "role": "user",
            "content": verification_prompt
        }],
        temperature=0,  # Use 0 for deterministic output
        max_tokens=50
    )

    # Simplifying the response check
    text_response = response.choices[0].message.content.strip().lower()
    if "correct" in text_response:
        return "correct"
    elif "incorrect" in text_response:
        return "incorrect"
    else:
        return "undetermined"  # Use this if the response isn't clear


**(1) df1: processed_60900_61899.csv**

In [25]:
# test
verify_llm_output(processed_df1.iloc[0]['Tweet'],processed_df1.iloc[0]['Travel Mode'], processed_df1.iloc[0]['Sentiment'], processed_df1.iloc[0]['Reasons'])

'correct'

In [27]:
processed_df1['Verification Result'] = processed_df1.apply(
    lambda row: verify_llm_output(row['Tweet'], row['Travel Mode'], row['Sentiment'], row['Reasons']), axis=1)

In [30]:
processed_df1

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons,Verification Result
0,60900,we already have congestion pricing for taxis &...,car,positive,congestion pricing is intended to encourage pe...,correct
1,60901,an the best cannot be that much i am sorry htt...,na,na,The tweet does not provide clear information r...,correct
2,60902,handicap uber's is getting out of hand,car,negative,The sentiment is negative towards Uber for the...,correct
3,60903,bitchh i would calmly get in a cab laugh ill b...,taxi,positive,The author expresses a positive sentiment towa...,correct
4,60904,take a cab. you are getting hepatitis walking ...,taxi,negative,The sentiment towards taking a taxi is negativ...,correct
...,...,...,...,...,...,...
995,61895,so...i confess ...i walked to target &amp; bou...,uber,positive,The sentiment towards the Uber ride is positiv...,correct
996,61896,"uber, ferry, plane, train today",plane,neutral,the tweet mentions various modes of transporta...,correct
997,61897,"hi, i order some food about a hour ago and i h...",food delivery,negative,delayed food delivery,correct
998,61898,i was there. it was an uber. none of the other...,taxi,neutral,the author mentions taking an Uber but doesn't...,correct


In [32]:
processed_df1.to_csv('Week 7 result_60900_61899((after self-verification).csv', index=False)

In [35]:
processed_df1['Verification Result'].value_counts().get('correct', 0)

999

**(2) df2: processed_177000_177999.csv**

In [ ]:
processed_df2['Verification Result'] = processed_df2.apply(
    lambda row: verify_llm_output(row['Tweet'], row['Travel Mode'], row['Sentiment'], row['Reasons']), axis=1)

In [31]:
processed_df2

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons,Verification Result
0,177000,please kick japan out of my country they are s...,car,negative,The sentiment towards the car transportation m...,correct
1,177001,once i learned about correctional exercises an...,car,neutral,The tweet metaphorically compares individuals ...,correct
2,177002,empty train cars &gt;&gt;,train,positive,The author expresses a positive sentiment towa...,correct
3,177003,that emerald parking lot is about 1 mile walk ...,car,negative,nightmare leaving the lot if you do not leave ...,correct
4,177004,"that is going to take time, i need car by end ...",car,positive,The author expresses the need for a car by the...,correct
...,...,...,...,...,...,...
995,177995,how about getting all trash off the sidewalk? ...,car,negative,The sentiment towards the car transportation m...,correct
996,177996,get trash off the sidewalk. trash corals in th...,car,negative,The author expresses frustration with trash co...,correct
997,177997,people of philadelphia find trae young's hotel...,car,negative,The sentiment is negative as the author expres...,correct
998,177998,ring rain coming back for more and do not you ...,car,positive,"The sentiment towards the car is positive, lik...",correct


In [33]:
processed_df2.to_csv('Week 7 result_177000_177999(after self-verification).csv', index=False)

In [36]:
processed_df2['Verification Result'].value_counts().get('correct', 0)

1000

## 6. Output Evaluation

In [2]:
df2_res=pd.read_csv('Week 7 result_177000_177999(after self-verification).csv')
df2_res

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons,Verification Result
0,177000,please kick japan out of my country they are s...,car,negative,The sentiment towards the car transportation m...,correct
1,177001,once i learned about correctional exercises an...,car,neutral,The comparison made in the tweet suggests a ne...,correct
2,177002,empty train cars &gt;&gt;,train,positive,The author expresses a positive sentiment towa...,correct
3,177003,that emerald parking lot is about 1 mile walk ...,car,negative,"The primary mode of transportation is by car, ...",correct
4,177004,"that is going to take time, i need car by end ...",car,neutral,the tweet indicates a need for a car by the en...,correct
...,...,...,...,...,...,...
995,177995,how about getting all trash off the sidewalk? ...,car,negative,The sentiment towards the car is negative due ...,correct
996,177996,get trash off the sidewalk. trash corals in th...,car,negative,The author expresses a negative sentiment towa...,correct
997,177997,people of philadelphia find trae young's hotel...,car,negative,The sentiment expressed is negative as the twe...,correct
998,177998,ring rain coming back for more and do not you ...,car,positive,The sentiment towards the car ride seems posit...,correct


In [5]:
def analyze_data(df):
    # Analyze 'Travel Mode'
    unique_travel_modes = df['Travel Mode'].unique()
    print("Distinct outcomes for Travel Mode:", unique_travel_modes)

    number_of_distinct_travel_modes = df['Travel Mode'].nunique()
    print("Number of distinct outcomes for Travel Mode:", number_of_distinct_travel_modes)

    travel_mode_counts = df['Travel Mode'].value_counts()
    print("Counts for each distinct outcome in Travel Mode:", travel_mode_counts)

    # Analyze 'Sentiment'
    unique_sentiments = df['Sentiment'].unique()
    print("Distinct outcomes for Sentiment:", unique_sentiments)

    number_of_distinct_sentiments = df['Sentiment'].nunique()
    print("Number of distinct outcomes for Sentiment:", number_of_distinct_sentiments)

    sentiment_counts = df['Sentiment'].value_counts()
    print("Counts for each distinct outcome in Sentiment:", sentiment_counts)

    # Calculate various rates
    extraction_rate = len(df[~df['Travel Mode'].isin(['na', 'NA'])]) / len(df)
    satisfaction_rate = len(df[df['Sentiment'] == 'positive']) / len(df)
    self_verification_rate = len(df[df['Verification Result'] == 'correct']) / len(df)
    
    if 'Cross Verification Result' in df.columns:
        cross_verification_rate = len(df[df['Cross Verification Result'] == 'correct']) / len(df)
    else:
        cross_verification_rate = None
    
    return {
        'Extraction Rate': extraction_rate,
        'Satisfaction Rate': satisfaction_rate,
        'Self Verification Rate': self_verification_rate
    }


In [6]:
analyze_data(df2_res)

Distinct outcomes for Travel Mode: ['car' 'train' 'na' 'bike' 'taxi' 'walking' 'subway' 'scooter' 'flight'
 nan 'helicopter' 'motorcycle' 'flying cars' 'skates' 'plane'
 'pedestrians' 'transit' 'boat' 'bus' 'horse']
Number of distinct outcomes for Travel Mode: 19
Counts for each distinct outcome in Travel Mode: car            867
na              48
bike            18
subway          18
train           11
walking          8
scooter          6
taxi             4
plane            3
bus              2
helicopter       2
flying cars      1
boat             1
pedestrians      1
flight           1
skates           1
motorcycle       1
transit          1
horse            1
Name: Travel Mode, dtype: int64
Distinct outcomes for Sentiment: ['negative' 'neutral' 'positive' 'na' 'mixed' 'potentially negative'
 'surprise']
Number of distinct outcomes for Sentiment: 7
Counts for each distinct outcome in Sentiment: negative                557
positive                260
neutral                 158
na 

{'Extraction Rate': 0.952,
 'Satisfaction Rate': 0.26,
 'Self Verification Rate': 1.0}